In [1]:
import argparse
import os
import utils
import json
import time
import requests

AAI_API_KEY = os.getenv("AAI_API_KEY")

def transcribe(audio_file):
    if AAI_API_KEY is None:
        raise RuntimeError("AAI_API_KEY environment variable not set. Try setting it now.")

    # Create header with authorization along with content-type
    header = {
        'authorization': AAI_API_KEY,
        'content-type': 'application/json'
    }

    upload_url = utils.upload_file(audio_file, header)

    # Request a transcription
    transcript_response = utils.request_transcript(upload_url, header)

    # Create a polling endpoint that will let us check when the transcription is complete
    polling_endpoint = utils.make_polling_endpoint(transcript_response)

    # Wait until the transcription is complete
    while True:
        polling_response = requests.get(polling_endpoint, headers=header)
        polling_response = polling_response.json()

        if polling_response['status'] == 'completed':
            break

        time.sleep(5)


    # Save and print transcript
    with open('transcript.json', 'w') as f:
        f = json.dumps(polling_response)

    return polling_response, polling_endpoint


In [4]:
header = {
        'authorization': AAI_API_KEY,
        'content-type': 'application/json'
    }

In [5]:
transcript_response, polling_endpoint = transcribe('audio.mp3')

KeyboardInterrupt: 

In [6]:
paragraphs = utils.get_paragraphs(polling_endpoint, header)

In [7]:
for p in paragraphs:
    print(p['text'])

AssemblyAI is a deep learning company that builds powerful APIs to help you transcribe and understand audio. The most common use case for the API is to automatically convert prerecorded audio and video files, as well as real time audio streams into text transcriptions. Our APIs convert audio and video into text using powerful deep learning models that we research and develop end to end in house. Millions of podcasts, zoom recordings, phone calls, or video files are being transcribed with AssemblyAI every single day. But where AssemblyAI really excels is with helping you understand your data.
So let's say we transcribe Joe Biden's State of the Union using assembly. AI's API. With our Auto Chapter feature, you can generate time coded summaries of the key moments of your audio file. For example, with the State of the Union address, we get chapter summaries like this auto Chapters automatically segments your audio or video files into chapters and provides a summary for each of these chapte

In [8]:
import cohere
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
co = cohere.Client(COHERE_API_KEY)

In [47]:
from nltk.tokenize import sent_tokenize
def create_embeds(texts, user_id="test"):
    texts = sent_tokenize(texts)
    embeds = co.embed(texts, model="large", truncate="right")
    return texts, embeds
texts = " ".join([p["text"] for p in paragraphs])
texts, embeds = create_embeds(texts)

In [10]:
len(embeds.embeddings)

23

In [58]:
from transformers import pipeline
from scipy.spatial.distance import cosine
import numpy as np
classifier = pipeline("zero-shot-classification",
                      model="valhalla/distilbart-mnli-12-1")
built_ins = ["entrepreneurship", "confidence", "Growth Mindset"]

def score_BART_text(text, model_name):
    pred = classifier(text, [model_name, "not " + model_name ])
    if pred[model_name] > pred["not " + model_name]:
        return 1
    else:
        return 0

def score_CUSTOM_text(text_embed, custom_model_embeds):
    mean_score = np.mean([cosine(e, text_embed) for e in custom_model_embeds.embeddings])
    if len(custom_model_embeds.embeddings) < 5:
        threshold = 1.0 - len(custom_model_embeds.embeddings) / 10
    else:
        threshold = 0.5
    if mean_score > threshold:
        return 1
    else:
        return 0
    

In [59]:
import pandas as pd
prefilled_models = {
    "Craftpersonship": "Find meaning in what we do through crafting excellence.",
    "Playfulness": "Great ideas come from health and happiness.",
    "Grit": "Perseverance driven by determination and passion.",
    "Empathy": "Innovation starts with understanding.",
    "Zest": "What sets you apart makes us unique.",
    "Courage": "Dare often and greatly."
}
texts_df = pd.DataFrame({"text":texts, "embeddings": embeds.embeddings})

In [60]:
def score_custom_models(text_df, customs_models=prefilled_models):
    for model_name in custom_models:
        _, embeds_custom = create_embeds(custom_models[model_name], user_id="test")
        texts_df[model_name] = texts_df["embeddings"].apply(lambda x: score_CUSTOM_text(x, embeds_custom))
        time.sleep(10) #Throttle since we're using the free tier for cohere
    return text_df

,text,embeddings,craftpersonship,Playfulness,Grit,Empathy,Zest,Courage
0,AssemblyAI is a deep learning company that bui...,"[1.3828125, 0.28051758, 1.9667969, -1.0039062,...",0,1,0,0,0,0
1,The most common use case for the API is to aut...,"[0.35839844, 1.578125, 2.1542969, 0.91064453, ...",1,1,1,1,1,1
2,Our APIs convert audio and video into text usi...,"[0.7441406, 0.6464844, 1.5029297, -1.5751953, ...",0,0,0,0,0,0
3,"Millions of podcasts, zoom recordings, phone c...","[0.24597168, 1.2382812, 2.2871094, 0.37695312,...",0,0,1,0,0,0
4,But where AssemblyAI really excels is with hel...,"[1.0498047, 1.7324219, 1.7011719, -0.0770874, ...",0,0,0,0,0,0


In [61]:
for model in built_ins:
     texts_df[model] = texts_df["text"].apply(lambda x: score_BART_text(x, model))

KeyError: 'entrepreneurship'

In [ ]:
pred = classifier(texts, [model_name, "not " + model_name ])